# TensorFlow Lite Conversion

In [1]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
import tensorflow as tf
import os
import logging
import numpy as np
from pathlib import Path
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from layers import ArcMarginPenaltyLogists
from losses import softmax_loss
from glob import glob
import dataset

tf.get_logger().setLevel(logging.ERROR)
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
!echo $CUDA_VISIBLE_DEVICES

1


In [3]:
### IJB-C Dataset

# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset

batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
head_type = 'ArcHead'
backbone_type = 'MobileNetV2' # MobileNetV2, ResNet50, EfficientNetLite0~6
w_decay=5e-4
num_classes = 85742 
dataset_len = 5822653 
base_lr = 0.01 # initially 0.01
epochs = 20
save_steps = 1000
train_size = int(0.8 * dataset_len)
steps_per_epoch = train_size // batch_size
val_size = dataset_len - train_size
validation_steps = val_size // batch_size
steps = 1
is_ccrop=False
binary_img=True
is_Adam = False

dgx = True

In [5]:
from glob import glob
if dgx:
    base_dir = "/raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/"
#     base_dir = "/raid/workspace/jbpark/FaceRecognition/checkpoints/wo_tfidentity/"
#     save_name = "ms1m_mobilenet_check/SGD/*"
    save_name = "ms1m_efficientnet_check/SGD/*"
else:
    base_dir = "/hd/jbpark/models/checkpoints/w_tfidentity/"
    save_name = "ms1m_check/SGD/*"
file_list = []
for files in glob(base_dir+save_name):
    file_list.append(files.split('/')[-1].split('l_')[-1])
file_list.sort()

load_file_name = []
for files in glob(base_dir+save_name):
    if file_list[0] == files.split('/')[-1].split('l_')[-1]:
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(initial_epoch)
print(best_checkpoint)

3
/raid/workspace/jbpark/FaceRecognition/checkpoints/w_tfidentity/ms1m_efficientnet_check/SGD/e_4_l_22.562232971191406.ckpt


In [6]:
accuracy = file_list[0].split('.')[0]
print(accuracy)

22


### Without Quantization

In [6]:
weight_file = best_checkpoint

model = ArcFaceModel(size=input_size,
                         backbone_type=backbone_type,
                         training=False)
model.load_weights(weight_file)

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TF Lite model.
if dgx:
    Path('/raid/workspace/jbpark/FaceRecognition/tflite/').mkdir(parents=True, exist_ok=True)
    with tf.io.gfile.GFile('/raid/workspace/jbpark/FaceRecognition/tflite/ms1m_acc_'+str(accuracy)+'.tflite', 'wb') as f:
        f.write(tflite_model)
else:
    Path('/hd/jbpark/models/tflite/').mkdir(parents=True, exist_ok=True)
    with tf.io.gfile.GFile('/hd/jbpark/models/tflite/ms1m_acc_'+str(accuracy)+'.tflite', 'wb') as f:
        f.write(tflite_model)

### With Qunatization

In [7]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'
if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'


train_dataset = dataset.load_representative_dataset(
    tfrecord_name, batch_size=1, train_size=train_size, binary_img=binary_img,
    is_ccrop=is_ccrop)

In [8]:
def representative_data_gen():
    count=0
    for input_value in train_dataset:
        count+=1
        if count > 100: return
        yield [input_value]

In [9]:
for data in representative_data_gen():
    print(data)

[<tf.Tensor: shape=(1, 112, 112, 3), dtype=float32, numpy=
array([[[[0.23220558, 0.22742265, 0.2081228 ],
         [0.14046232, 0.13567941, 0.1165477 ],
         [0.12379567, 0.11901274, 0.10275826],
         ...,
         [0.21412961, 0.21143922, 0.20187336],
         [0.22984652, 0.22844528, 0.22028066],
         [0.28368843, 0.28787348, 0.28578094]],

        [[0.20250277, 0.19771984, 0.17632745],
         [0.11654995, 0.11176702, 0.09224297],
         [0.12980056, 0.12501764, 0.10588593],
         ...,
         [0.2243165 , 0.21953356, 0.20996772],
         [0.23971172, 0.23582558, 0.22805333],
         [0.2776866 , 0.2776866 , 0.2776866 ]],

        [[0.15916638, 0.15438345, 0.13046882],
         [0.09868843, 0.09390551, 0.073877  ],
         [0.1336149 , 0.12883198, 0.10970027],
         ...,
         [0.221023  , 0.21624006, 0.2066742 ],
         [0.25092494, 0.24703883, 0.23926656],
         [0.28882325, 0.28882325, 0.28882325]],

        ...,

        [[0.11896855, 0.17158073,

In [10]:
weight_file = best_checkpoint

model = ArcFaceModel(size=input_size,
                         backbone_type=backbone_type,
                         training=False)
model.load_weights(weight_file)

# Convert the model.
model.input.set_shape((1,) + model.input.shape[1:])
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

# Save the TF Lite model.
if dgx:
    Path('/raid/workspace/jbpark/FaceRecognition/tflite/').mkdir(parents=True, exist_ok=True)
    with tf.io.gfile.GFile('/raid/workspace/jbpark/FaceRecognition/tflite/ms1m_acc_'+str(accuracy)+'.tflite', 'wb') as f:
        f.write(tflite_model_quant)
else:
    Path('/hd/jbpark/models/tflite/').mkdir(parents=True, exist_ok=True)
    with tf.io.gfile.GFile('/hd/jbpark/models/tflite/ms1m_acc_'+str(accuracy)+'.tflite', 'wb') as f:
        f.write(tflite_model_quant)

### Input / Output Check

In [7]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors.
if dgx:
    interpreter = tf.lite.Interpreter(model_path='/raid/workspace/jbpark/FaceRecognition/tflite/ms1m_acc_'+str(accuracy)+'.tflite')
else:
    interpreter = tf.lite.Interpreter(model_path='/hd/jbpark/models/tflite/ms1m_acc_'+str(accuracy)+'.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
input_details

[{'name': 'input_image',
  'index': 0,
  'shape': array([  1, 112, 112,   3], dtype=int32),
  'shape_signature': array([ -1, 112, 112,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [8]:
output_details = interpreter.get_output_details()
output_details

[{'name': 'Identity',
  'index': 180,
  'shape': array([  1, 512], dtype=int32),
  'shape_signature': array([ -1, 512], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]